# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 2, Evaluation

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
wmg.Init("LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerfSer'.


In [3]:
// Manually add database (e.g. when accessing the database of user `JenkinsCI` as some other, regular guy)
//var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

In [4]:
databases

#0: { Session Count = 79; Grid Count = 30; Path = \\fdygitrunner\ValidationTests\LinslvPerfSer }


In [5]:
wmg.Sessions

#0: LinslvPerfSer	XdgStokes-J4096_p5_classic_pardiso	03/15/2022 20:33:08	4a1f70fb...
#1: LinslvPerfSer	XdgStokes-J13824_p3_classic_pardiso	03/15/2022 20:31:47	8a5c0f7d...
#2: LinslvPerfSer	XdgStokes-J13824_p2_classic_pardiso	03/15/2022 20:29:44	bdad41bd...
#3: LinslvPerfSer	XdgStokes-J4096_p3_classic_pardiso	03/15/2022 20:31:06	48fd5d03...
#4: LinslvPerfSer	XdgStokes-J512_p5_classic_pardiso	03/15/2022 20:32:30	598d496f...
#5: LinslvPerfSer	XdgStokes-J4096_p2_classic_pardiso	03/15/2022 20:29:08	989f8cae...
#6: LinslvPerfSer	XdgStokes-J512_p3_classic_pardiso	03/15/2022 20:30:24	16d8e8f5...
#7: LinslvPerfSer	XdgStokes-J512_p2_classic_pardiso	03/15/2022 20:28:32	5ad7e2fc...
#8: LinslvPerfSer	BottiPietroStokes2D-J16384_p5_classic_pardiso	03/15/2022 20:17:52	a74031a4...
#9: LinslvPerfSer	BottiPietroStokes2D-J4096_p5_classic_pardiso	03/15/2022 20:17:14	eb570035...
#10: LinslvPerfSer	BottiPietroStokes2D-J36864_p3_classic_pardiso	03/15/2022 20:13:38	2141939f...
#11: LinslvPerfSer	BottiPietroSto

In [6]:
//var s = wmg.Sessions.Single(s => s.Name.Contains("J20480_k3_exp_Kcycle_schwarz"));
//s.DeployPath

In [7]:
using System.IO;
using BoSSS.Application.SipPoisson;

In [16]:
var SIstokes = wmg.Sessions[0];
var SIpisson = wmg.Sessions[69];

In [39]:
SIstokes.GetControl().GetSolverType().Name

XNSE`1

In [80]:
var SI = SIstokes;
SI.DeployPath

\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar15_203243

In [35]:
//foreach(var kv in SIstokes.KeysAndQueries) {
//Console.WriteLine($"{kv.Key}    {kv.Value}");
//}

index,value
0,DGdegree:Velocity*


In [28]:
SIstokes.GetSessionDirectory()

\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\4a1f70fb-d612-4d9b-9ae8-8af3e4b6d6f1

## Create Table for Post-Processing

The timeing information that is requires for this study is not pesent in the default session table.
Instead, it must be extracted from the *session profiling*.

To obtain timing-measurements, the instrumentation generated by certain `BlockTrace`-blocks within the `UniSolver` class is extracted and added to the data table:

In [70]:
// evaluators to add additional columns to the session table
static class AddCols {
    static bool IsStokes(ISessionInfo SI) {
        return (SI.GetControl().GetSolverType().Name.Contains("XNSE"));
    }
    
    
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        ilPSP.Tracing.MethodCallRecord nd;
        if(IsStokes(SI))
            nd = mcr.FindChildren("*.XdgTimestepping.ComputeOperatorMatrix").Single();
        else 
            nd  = mcr.FindChildren("MatrixAssembly").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Aggregation_basis_init").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Init").Single();
        //var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Run").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
    }
    static public object ComputeNodeName(ISessionInfo SI) {
        return SI.ComputeNodeNames.First();
    }
    
    static public object DGdegree(ISessionInfo SI) {
        if(IsStokes(SI)) {
            return SI.KeysAndQueries.Single(kv => kv.Key.Contains("DGdegree:Velocity")).Value;
        } else {
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:T").Value;
        }
    }
}

In [73]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
wmg.AdditionalSessionTableColums.Add("ComputeNodeName", AddCols.ComputeNodeName);
wmg.AdditionalSessionTableColums.Add("DGdegree", AddCols.DGdegree);

In [74]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

In [75]:
//wmg.Sessions[0].KeysAndQueries["NoIter"].GetType()

In [76]:
//wmg.Sessions[0].Name

In [77]:
//SessTab.GetColumn("LinearSolver.Name")

Select those columns which are of interest:

In [78]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree", "Grid:NoOfCells", "LinearSolver.Name", "LinearSolver.Shortname", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter", "ComputeNodeName");

In [52]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

In [ ]:
//SubTab.Print();

## Vizualisation of Results

The following data is available:

In [ ]:
SubTab.GetColumnNames()

Available DG degrees:

In [ ]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree:T").ToSet().OrderBy(s => s).ToArray();
DGdegrees

All used solvers:

In [ ]:
SubTab.GetColumn<string>("LinearSolver.Shortname").ToSet()

In [ ]:
//RuntimePlot.PlotNow();

In [ ]:
//using SolverCodes = BoSSS.Solution.Control.LinearSolverConfig.Code;

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [ ]:
PlotFormat SlvCode2Pltfmt(string solver_name) { 
    var Fmt = new PlotFormat();
    switch(solver_name) {
        case "PARDISO": 
            Fmt.PointType = PointTypes.OpenCircle;
            Fmt.DashType  = DashTypes.Dotted;
            break;
        case "GMRES w p2G": 
            Fmt.PointType = PointTypes.Box;
            break;
        case "OrthoMG w Add Swz": 
            Fmt.PointType = PointTypes.LowerTriangle;
            break;
        case "OrthoMG w ILU":
            Fmt.PointType = PointTypes.Diamond;
            break;
        default:
            Console.WriteLine("unknown: " + solver_name); 
            break;
    } 
    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.5;
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = LineColors.Black;
    Fmt.LineWidth = 3;
    return Fmt;
}

In [ ]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax) {

int rows    = DGdegrees.Length;
int columns = 1;
string[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree:T"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           string solver_name = (string) (Row["LinearSolver.Shortname"]);
           //ignore the solvers specified in ingore_solvers
           foreach(string sc in ignore_solvers){
               if(solver_name == sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           
           //process the other solvers
           Fmt = SlvCode2Pltfmt(solver_name);
           Nmn = solver_name;
       });
    
       double[] dof = new[]{1e3,1e6};
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e7;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 5;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == 0) {
          multiplots[iRow,iCol].ShowLegend = true;
          multiplots[iRow,iCol].LegendAlignment = new string[]{"i","t","l"};
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

### Total Runtime

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [ ]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

In [ ]:
multiplots[0,0].Regression()

In [ ]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[0,0].Regression().Single(tt =>  tt.Key == "GMRES w p2G").Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 2");

In [ ]:
multiplots[1,0].Regression()

In [ ]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[1,0].Regression().Single(tt =>  tt.Key == "GMRES w p2G").Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 3");

In [ ]:
multiplots[2,0].Regression()

In [ ]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[2,0].Regression().Single(tt =>  tt.Key == "GMRES w p2G").Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 5");

Experimental: Latex Preview:

In [ ]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
multiplots.PlotCairolatex().PlotNow()

### Number of Iterations

For the iterative solvers, obe would expect **almost constant number of solver iterations**, 
independent of grid resolution. It may, however depend in nonlinar fashion on the DG polynomial order.

In [ ]:
var multiplots = PlotSolverBehave("NoIter", false, 0, 140);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

In [ ]:
multiplots[0,0].dataGroups

In [ ]:
multiplots[1,0].dataGroups

In [ ]:
multiplots[2,0].dataGroups

### Runtime per Iteration

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [ ]:
var multiplots = PlotSolverBehave("Solver_TimePerIter", true, 1e-2, 1e+3);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

In [ ]:
multiplots[0,0].Regression()

In [ ]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[0,0].Regression().Single(tt =>  tt.Key == "GMRES w p2G").Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 2");
NUnit.Framework.Assert.LessOrEqual(
   multiplots[0,0].Regression().Single(tt =>  tt.Key == "OrthoMG w Add Swz").Value,
   1.1,
   "Ortho MG scaling out of linear for p = 2");

In [ ]:
multiplots[1,0].Regression()

In [ ]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[1,0].Regression().Single(tt =>  tt.Key == "GMRES w p2G").Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 3");
NUnit.Framework.Assert.LessOrEqual(
   multiplots[1,0].Regression().Single(tt =>  tt.Key == "OrthoMG w Add Swz").Value,
   1.1,
   "Ortho MG scaling out of linear for p = 3");

In [ ]:
multiplots[2,0].Regression()

In [ ]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[2,0].Regression().Single(tt =>  tt.Key == "GMRES w p2G").Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 5");
NUnit.Framework.Assert.LessOrEqual(
   multiplots[2,0].Regression().Single(tt =>  tt.Key == "OrthoMG w Add Swz").Value,
   1.2,
   "Ortho MG scaling out of linear for p = 5");

## Appendix: Table Form of Extracted Data

In [ ]:
SubTab.Print()